## TactiTrack


In [ ]:
import cv2 
import numpy as np 
import os 

from ultralytics import YOLO
import supervision as sv 
from classifiers import *
from sklearn.cluster import KMeans
from collections import defaultdict
from classifiers import *
from visualization import *
from sklearn.cluster import KMeans
from interpolations import *




In [11]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
from collections import defaultdict
from sklearn.cluster import KMeans
import supervision as sv
from classifiers import *
from visualization import *
from utils import *

# paths
input_video_path = 'input_videos/9.mp4'
output_video_path = 'output_videos/output.mp4'
model_path = 'models/best.pt'

# making video and readung 
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

model = YOLO(model_path)
tracker = sv.ByteTrack()


# possession variables
team_possession = {0: 0, 1: 0}
current_possession = None
ball_holder = None
ball_hold_frames = 0
possession_threshold = 5
total_frames = 0

# additional variables
reference_colors = None
team_assignments = {}
color_history = defaultdict(list)
ball_positions_history = []
max_history_length = 10

# Team colors
TEAM_COLORS = {
    0: (0, 0, 255),  # Team 1 - Red
    1: (255, 0, 0),  # Team 2 - Blue
    "goalkeeper": (0, 0, 255),
    "referee": (0, 255, 255),
    "ball": (255, 255, 255)
}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1 detection 
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    players = detections[detections.class_id == 2]
    goalkeepers = detections[detections.class_id == 1]
    referees = detections[detections.class_id == 3]
    ball = detections[detections.class_id == 0]

    # Step 2 classification based on jerse color 
    if len(players) > 0:
        grass_color = get_grass_color(frame)
        grass_hsv = cv2.cvtColor(np.uint8([[grass_color]]), cv2.COLOR_BGR2HSV)[0, 0]
        jersey_colors = [
            get_jersey_color(frame[int(y1):int(y2), int(x1):int(x2)], grass_hsv)
            for x1, y1, x2, y2 in players.xyxy
        ]
        
        # detect the team colors
        if len(jersey_colors) >= 2 and not reference_colors:
            kmeans = KMeans(n_clusters=2).fit(jersey_colors)
            reference_colors = {
                0: kmeans.cluster_centers_[0],
                1: kmeans.cluster_centers_[1],
            }

        # ربط اللاعبين بفرقهم
        teams = []
        for color in jersey_colors:
            if reference_colors:
                distances = [np.linalg.norm(color - ref) for ref in reference_colors.values()]
                team = distances.index(min(distances))
                teams.append(team)

    # Step 3 tracking 
    players = tracker.update_with_detections(players)

    for i, track_id in enumerate(players.tracker_id):
        if i < len(teams):
            color_history[track_id].append(teams[i])
            if len(color_history[track_id]) >= 5:
                assigned_team = max(set(color_history[track_id]), key=color_history[track_id].count)
                team_assignments[track_id] = assigned_team

    # Step 4 ball position interpolation 
    ball_bbox = None
    if len(ball.xyxy) > 0:
        ball_bbox = tuple(map(int, ball.xyxy[0]))
        ball_positions_history.append(ball_bbox)
    else:
        ball_positions_history.append(None)

    if len(ball_positions_history) > max_history_length:
        ball_positions_history.pop(0)

    if None in ball_positions_history:
        interpolated = interpolate_ball_positions(ball_positions_history)
        if interpolated[-1]:
            ball_bbox = interpolated[-1]

    #Step 5  calculate possession 
    closest_player = None
    min_distance = float('inf')

    if ball_bbox:
        ball_center = get_center_of_bbox(ball_bbox)
        for i, track_id in enumerate(players.tracker_id):
            player_center = get_center_of_bbox(players.xyxy[i])
            dist = np.linalg.norm(np.array(ball_center) - np.array(player_center))
            if dist < min_distance and dist < 40:
                min_distance = dist
                closest_player = track_id

        if closest_player:
            if closest_player == ball_holder:
                ball_hold_frames += 1
            else:
                ball_holder = closest_player
                ball_hold_frames = 1

            if ball_hold_frames >= possession_threshold:
                new_possession = team_assignments.get(ball_holder, None)
                if new_possession is not None:
                    current_possession = new_possession
                    team_possession[current_possession] += 1

    total_frames += 1

    # Step 6 drawing 
    annotated = frame.copy()

    # draw players
    for bbox, track_id in zip(players.xyxy, players.tracker_id):
        team = team_assignments.get(track_id, 0)
        has_ball = (track_id == ball_holder and ball_hold_frames >= possession_threshold)
        annotated = draw_annotation(annotated, bbox, TEAM_COLORS[team], track_id, team, has_ball)

    # draw ball
    if ball_bbox:
        draw_triangle(annotated, ball_bbox, TEAM_COLORS["ball"])

    # Draw possession
    if total_frames > 0:
        team_0_ratio = team_possession[0] / total_frames
        team_1_ratio = team_possession[1] / total_frames
        total_ratio = team_0_ratio + team_1_ratio
        team_0_possession = (team_0_ratio / total_ratio) * 100 if total_ratio > 0 else 50
        team_1_possession = 100 - team_0_possession

        possession_text = f"Team {current_possession + 1} has possession" if current_possession is not None else "No possession"
        annotated = add_transparent_rectangle(annotated, width - 400, height - 130, width - 10, height - 10, (192, 192, 192), 0.7)
        cv2.putText(annotated, f"Team 1: {team_0_possession:.1f}%", (width - 350, height - 100), cv2.FONT_HERSHEY_SIMPLEX, 1, TEAM_COLORS[0], 2)
        cv2.putText(annotated, f"Team 2: {team_1_possession:.1f}%", (width - 350, height - 60), cv2.FONT_HERSHEY_SIMPLEX, 1, TEAM_COLORS[1], 2)
        cv2.putText(annotated, possession_text, (width - 350, height - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

 
    out.write(annotated)

cap.release()
out.release()
print("✅ Processing complete.")
os.system(f'start {output_video_path}')



0: 384x640 2 goalkeepers, 17 players, 4 referees, 42.8ms
Speed: 6.1ms preprocess, 42.8ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 goalkeepers, 20 players, 3 referees, 38.2ms
Speed: 4.4ms preprocess, 38.2ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 goalkeepers, 20 players, 3 referees, 37.2ms
Speed: 3.4ms preprocess, 37.2ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 goalkeepers, 18 players, 3 referees, 37.3ms
Speed: 3.8ms preprocess, 37.3ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 goalkeepers, 17 players, 2 referees, 37.5ms
Speed: 4.4ms preprocess, 37.5ms inference, 3.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 goalkeepers, 18 players, 2 referees, 37.5ms
Speed: 2.8ms preprocess, 37.5ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 goalkeepers, 19 players, 1 referee, 37.5ms
Speed: 3.4ms 

0

In [16]:
os.system('start output_videos/output.mp4')


0

In [ ]:
	import cv2
	import os 
	from ultralytics import YOLO

	# تحميل موديل pose
	model = YOLO('models/pitch_model.pt')  # اختار الموديل اللي شغال عندك

	# فتح ملف الفيديو\
	
	cap = cv2.VideoCapture("input_videos/2.mp4")  # غيّر للمسار الحقيقي للفيديو
	fps = cap.get(cv2.CAP_PROP_FPS)
	width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
	height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
	out = cv2.VideoWriter('output_videos/output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))


	while cap.isOpened():
			ret, frame = cap.read()
			if not ret:
					break

			# تشغيل الموديل على الفريم
			results = model(frame)[0]

			if results.keypoints is not None:
					for kpts in results.keypoints.data:
							for i, (x, y, conf) in enumerate(kpts):
									x, y, conf = float(x), float(y), float(conf)
									if conf > 0.5:
											cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)
											
											cv2.putText(frame, str(i), (int(x) + 5, int(y) - 5),
																	cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
											
								

			out.write(frame)

	cap.release()
	out.release()
	cv2.destroyAllWindows()
	os.system('start output_videos/output.mp4')



0: 384x640 1 pitch, 176.5ms
Speed: 11.0ms preprocess, 176.5ms inference, 207.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 146.5ms
Speed: 4.2ms preprocess, 146.5ms inference, 7.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 146.4ms
Speed: 3.4ms preprocess, 146.4ms inference, 4.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 153.0ms
Speed: 3.5ms preprocess, 153.0ms inference, 7.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 163.5ms
Speed: 4.2ms preprocess, 163.5ms inference, 4.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 153.6ms
Speed: 2.8ms preprocess, 153.6ms inference, 4.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 145.1ms
Speed: 2.3ms preprocess, 145.1ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 147.5ms
Speed: 3.5ms preprocess, 147.5ms inference, 7.6ms postprocess per image at shap

0

In [17]:
# Add this function to detect possible pass receivers
def find_available_passes(frame, players, ball_holder, team_assignments):
    """
    Find all possible pass recipients for the player with the ball.
    Returns a list of (receiver_id, distance, is_clear) tuples.
    """
    if ball_holder is None or ball_holder not in team_assignments:
        return []
    
    holder_team = team_assignments[ball_holder]
    holder_idx = None
    for i, track_id in enumerate(players.tracker_id):
        if track_id == ball_holder:
            holder_idx = i
            break
    
    if holder_idx is None:
        return []
    
    holder_position = get_center_of_bbox(players.xyxy[holder_idx])
    available_passes = []
    
    # Find teammates
    for i, track_id in enumerate(players.tracker_id):
        if track_id != ball_holder and track_id in team_assignments and team_assignments[track_id] == holder_team:
            receiver_position = get_center_of_bbox(players.xyxy[i])
            distance = np.linalg.norm(np.array(holder_position) - np.array(receiver_position))
            
            # Check if pass is clear (no opponents in the way)
            is_clear = True
            for j, other_id in enumerate(players.tracker_id):
                if other_id in team_assignments and team_assignments[other_id] != holder_team:
                    opponent_position = get_center_of_bbox(players.xyxy[j])
                    if is_point_on_line_segment(holder_position, receiver_position, opponent_position, tolerance=30):
                        is_clear = False
                        break
            
            available_passes.append((track_id, distance, is_clear))
    
    return available_passes

# Add this helper function to check if a point is between two other points
def is_point_on_line_segment(p1, p2, p3, tolerance=20):
    """
    Check if point p3 is on the line segment from p1 to p2 within a tolerance.
    """
    # Convert to numpy arrays for easier operations
    p1 = np.array(p1)
    p2 = np.array(p2)
    p3 = np.array(p3)
    
    # Calculate distance from p3 to the line segment from p1 to p2
    line_length = np.linalg.norm(p2 - p1)
    if line_length == 0:
        return np.linalg.norm(p3 - p1) <= tolerance
    
    # Project p3 onto the line
    t = np.dot(p3 - p1, p2 - p1) / (line_length * line_length)
    
    # Check if projection is on the segment
    if 0 <= t <= 1:
        projection = p1 + t * (p2 - p1)
        distance = np.linalg.norm(p3 - projection)
        return distance <= tolerance
    
    return False

# Function to draw pass lines
def draw_pass_lines(frame, players, available_passes, team_color):
    """
    Draw lines representing available passes on the frame.
    Green lines for clear passes, red lines for blocked passes.
    """
    holder_idx = None
    for i, track_id in enumerate(players.tracker_id):
        if track_id == ball_holder:
            holder_idx = i
            break
    
    if holder_idx is None:
        return frame
    
    holder_position = get_center_of_bbox(players.xyxy[holder_idx])
    
    for receiver_id, distance, is_clear in available_passes:
        for i, track_id in enumerate(players.tracker_id):
            if track_id == receiver_id:
                receiver_position = get_center_of_bbox(players.xyxy[i])
                color = (0, 255, 0) if is_clear else (0, 0, 255)  # Green for clear, red for blocked
                thickness = 2 if distance < 150 else 1  # Thicker lines for closer players
                
                # Draw the pass line
                cv2.line(frame, 
                    (int(holder_position[0]), int(holder_position[1])),
                    (int(receiver_position[0]), int(receiver_position[1])),
                    color, thickness)
                
                # Draw a small circle at the end to indicate the receiver
                cv2.circle(frame, 
                    (int(receiver_position[0]), int(receiver_position[1])),
                    5, color, -1)
    
    return frame

In [18]:
	# Add these functions to analyze zone control

def divide_field_into_zones(width, height, zones_x=3, zones_y=2):
    """
    Divide the field into a grid of zones.
    Returns a list of (x1, y1, x2, y2) coordinates for each zone.
    """
    zone_width = width // zones_x
    zone_height = height // zones_y
    zones = []
    
    for y in range(zones_y):
        for x in range(zones_x):
            x1 = x * zone_width
            y1 = y * zone_height
            x2 = (x + 1) * zone_width
            y2 = (y + 1) * zone_height
            zones.append((x1, y1, x2, y2))
    
    return zones

def get_player_zone(player_center, zones):
    """
    Determine which zone a player is in based on their position.
    Returns the zone index.
    """
    x, y = player_center
    for i, (x1, y1, x2, y2) in enumerate(zones):
        if x1 <= x <= x2 and y1 <= y <= y2:
            return i
    return None

def analyze_zone_control(players, team_assignments, zones):
    """
    Analyze which team controls each zone.
    Returns a dictionary mapping zone index to (team1_count, team2_count, dominant_team).
    """
    zone_control = {}
    for i in range(len(zones)):
        zone_control[i] = {0: 0, 1: 0, "dominant": None}
    
    for i, track_id in enumerate(players.tracker_id):
        if track_id in team_assignments:
            player_center = get_center_of_bbox(players.xyxy[i])
            zone = get_player_zone(player_center, zones)
            if zone is not None:
                team = team_assignments[track_id]
                zone_control[zone][team] += 1
    
    # Determine dominant team for each zone
    for zone in zone_control:
        team1_count = zone_control[zone][0]
        team2_count = zone_control[zone][1]
        
        if team1_count > team2_count:
            zone_control[zone]["dominant"] = 0
        elif team2_count > team1_count:
            zone_control[zone]["dominant"] = 1
        else:
            zone_control[zone]["dominant"] = None  # Tie
    
    return zone_control

def draw_zone_control(frame, zones, zone_control, team_colors):
    """
    Visualize zone control by overlaying semi-transparent colors on the field.
    """
    overlay = frame.copy()
    
    for zone_idx, (x1, y1, x2, y2) in enumerate(zones):
        dominant_team = zone_control[zone_idx]["dominant"]
        team1_count = zone_control[zone_idx][0]
        team2_count = zone_control[zone_idx][1]
        
        # Draw zone outline
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
        
        # Add zone number
        cv2.putText(frame, f"Zone {zone_idx+1}", (x1+10, y1+25), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        # Determine overlay color based on dominant team
        if dominant_team is not None:
            color = team_colors[dominant_team]
            # Calculate opacity based on dominance
            total_players = team1_count + team2_count
            if total_players > 0:
                dominance_ratio = max(team1_count, team2_count) / total_players
                opacity = 0.2 + (0.3 * dominance_ratio)  # Between 0.2 and 0.5
            else:
                opacity = 0.2
                
            # Draw filled rectangle with team color
            cv2.rectangle(overlay, (x1, y1), (x2, y2), color, -1)
            
            # Add player count
            text = f"{team1_count}:{team2_count}"
            cv2.putText(overlay, text, (x1+10, y1+60), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Blend the overlay with the original frame
    cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)
    
    return frame

def add_zone_control_legend(frame, width, height, team_colors):
    """
    Add a legend explaining the zone control visualization
    """
    # Create a semi-transparent background for the legend
    legend_x1 = 10
    legend_y1 = 10
    legend_x2 = 250
    legend_y2 = 100
    frame = add_transparent_rectangle(frame, legend_x1, legend_y1, legend_x2, legend_y2, (0, 0, 0), 0.7)
    
    # Add title
    cv2.putText(frame, "Zone Control Legend:", (legend_x1 + 10, legend_y1 + 30), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    
    # Add team colors
    cv2.rectangle(frame, (legend_x1 + 10, legend_y1 + 40), (legend_x1 + 30, legend_y1 + 60), team_colors[0], -1)
    cv2.putText(frame, "Team 1", (legend_x1 + 40, legend_y1 + 55), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    
    cv2.rectangle(frame, (legend_x1 + 10, legend_y1 + 70), (legend_x1 + 30, legend_y1 + 90), team_colors[1], -1)
    cv2.putText(frame, "Team 2", (legend_x1 + 40, legend_y1 + 85), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    
    return frame

In [20]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
from collections import defaultdict
from sklearn.cluster import KMeans
import supervision as sv
from classifiers import *
from visualization import *
from utils import *
from interpolations import  *

# paths
input_video_path = 'input_videos/9.mp4'
output_video_path = 'output_videos/output.mp4'
model_path = 'models/best.pt'

# making video and readung 
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

model = YOLO(model_path)
tracker = sv.ByteTrack()
# Add these variables at the start of your script, with the other variables
zones = None
zone_control = None


# possession variables
team_possession = {0: 0, 1: 0}
current_possession = None
ball_holder = None
ball_hold_frames = 0
possession_threshold = 5
total_frames = 0

# additional variables
reference_colors = None
team_assignments = {}
color_history = defaultdict(list)
ball_positions_history = []
max_history_length = 10

# Team colors
TEAM_COLORS = {
    0: (0, 0, 255),  # Team 1 - Red
    1: (255, 0, 0),  # Team 2 - Blue
    "goalkeeper": (0, 0, 255),
    "referee": (0, 255, 255),
    "ball": (255, 255, 255)
}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1 detection 
    results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    players = detections[detections.class_id == 2]
    goalkeepers = detections[detections.class_id == 1]
    referees = detections[detections.class_id == 3]
    ball = detections[detections.class_id == 0]

    # Step 2 classification based on jerse color 
    if len(players) > 0:
        grass_color = get_grass_color(frame)
        grass_hsv = cv2.cvtColor(np.uint8([[grass_color]]), cv2.COLOR_BGR2HSV)[0, 0]
        jersey_colors = [
            get_jersey_color(frame[int(y1):int(y2), int(x1):int(x2)], grass_hsv)
            for x1, y1, x2, y2 in players.xyxy
        ]
        
        # detect the team colors
        if len(jersey_colors) >= 2 and not reference_colors:
            kmeans = KMeans(n_clusters=2).fit(jersey_colors)
            reference_colors = {
                0: kmeans.cluster_centers_[0],
                1: kmeans.cluster_centers_[1],
            }

        # ربط اللاعبين بفرقهم
        teams = []
        for color in jersey_colors:
            if reference_colors:
                distances = [np.linalg.norm(color - ref) for ref in reference_colors.values()]
                team = distances.index(min(distances))
                teams.append(team)

    # Step 3 tracking 
    players = tracker.update_with_detections(players)

    for i, track_id in enumerate(players.tracker_id):
        if i < len(teams):
            color_history[track_id].append(teams[i])
            if len(color_history[track_id]) >= 5:
                assigned_team = max(set(color_history[track_id]), key=color_history[track_id].count)
                team_assignments[track_id] = assigned_team

    # Step 4 ball position interpolation 
    ball_bbox = None
    if len(ball.xyxy) > 0:
        ball_bbox = tuple(map(int, ball.xyxy[0]))
        ball_positions_history.append(ball_bbox)
    else:
        ball_positions_history.append(None)

    if len(ball_positions_history) > max_history_length:
        ball_positions_history.pop(0)

    if None in ball_positions_history:
        interpolated = interpolate_ball_positions(ball_positions_history)
        if interpolated[-1]:
            ball_bbox = interpolated[-1]

    #Step 5  calculate possession 
    zones = divide_field_into_zones(width, height, zones_x=3, zones_y=2)
    annotated = frame.copy()

    # Add this inside your main loop, before drawing
    # Calculate zone control
    if len(players.tracker_id) > 0:
        zone_control = analyze_zone_control(players, team_assignments, zones)

    # Then add this to your drawing section:
# Draw zone control visualization (add this before drawing players)
    if zone_control:
        annotated = draw_zone_control(annotated, zones, zone_control, TEAM_COLORS)
        annotated = add_zone_control_legend(annotated, width, height, TEAM_COLORS)
        closest_player = None
        min_distance = float('inf')

    if ball_bbox:
        ball_center = get_center_of_bbox(ball_bbox)
        for i, track_id in enumerate(players.tracker_id):
            player_center = get_center_of_bbox(players.xyxy[i])
            dist = np.linalg.norm(np.array(ball_center) - np.array(player_center))
            if dist < min_distance and dist < 40:
                min_distance = dist
                closest_player = track_id

        if closest_player:
            if closest_player == ball_holder:
                ball_hold_frames += 1
            else:
                ball_holder = closest_player
                ball_hold_frames = 1

            if ball_hold_frames >= possession_threshold:
                new_possession = team_assignments.get(ball_holder, None)
                if new_possession is not None:
                    current_possession = new_possession
                    team_possession[current_possession] += 1

    total_frames += 1
    available_passes = []
    if ball_holder and ball_holder in team_assignments:
        available_passes = find_available_passes(frame, players, ball_holder, team_assignments)
    

    # Step 6 drawing 
    


    # draw players
    for bbox, track_id in zip(players.xyxy, players.tracker_id):
        team = team_assignments.get(track_id, 0)
        has_ball = (track_id == ball_holder and ball_hold_frames >= possession_threshold)
        annotated = draw_annotation(annotated, bbox, TEAM_COLORS[team], track_id, team, has_ball)

    # draw ball
    if ball_bbox:
        draw_triangle(annotated, ball_bbox, TEAM_COLORS["ball"])

    # Draw possession
    if total_frames > 0:
        team_0_ratio = team_possession[0] / total_frames
        team_1_ratio = team_possession[1] / total_frames
        total_ratio = team_0_ratio + team_1_ratio
        team_0_possession = (team_0_ratio / total_ratio) * 100 if total_ratio > 0 else 50
        team_1_possession = 100 - team_0_possession

        possession_text = f"Team {current_possession + 1} has possession" if current_possession is not None else "No possession"
        annotated = add_transparent_rectangle(annotated, width - 400, height - 130, width - 10, height - 10, (192, 192, 192), 0.7)
        cv2.putText(annotated, f"Team 1: {team_0_possession:.1f}%", (width - 350, height - 100), cv2.FONT_HERSHEY_SIMPLEX, 1, TEAM_COLORS[0], 2)
        cv2.putText(annotated, f"Team 2: {team_1_possession:.1f}%", (width - 350, height - 60), cv2.FONT_HERSHEY_SIMPLEX, 1, TEAM_COLORS[1], 2)
        cv2.putText(annotated, possession_text, (width - 350, height - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    if ball_holder and ball_holder in team_assignments:
        team = team_assignments[ball_holder]
        annotated = draw_pass_lines(annotated, players, available_passes, TEAM_COLORS[team],ball_holder=ball_holder)
    
    # Add pass statistics to the display (optional)
    if ball_holder and available_passes:
        clear_passes = sum(1 for _, _, is_clear in available_passes if is_clear)
        cv2.putText(annotated, 
                    f"Clear passes: {clear_passes}/{len(available_passes)}", 
                    (50, height - 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.8, (255, 255, 255), 2)
 
    out.write(annotated)

cap.release()
out.release()
print("✅ Processing complete.")
os.system(f'start {output_video_path}')



0: 384x640 2 goalkeepers, 17 players, 4 referees, 35.9ms
Speed: 2.3ms preprocess, 35.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 goalkeepers, 20 players, 3 referees, 34.4ms
Speed: 3.9ms preprocess, 34.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 goalkeepers, 20 players, 3 referees, 34.8ms
Speed: 2.3ms preprocess, 34.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 goalkeepers, 18 players, 3 referees, 34.6ms
Speed: 2.1ms preprocess, 34.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 goalkeepers, 17 players, 2 referees, 34.4ms
Speed: 2.1ms preprocess, 34.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 goalkeepers, 18 players, 2 referees, 35.2ms
Speed: 2.5ms preprocess, 35.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 goalkeepers, 19 players, 1 referee, 35.2ms
Speed: 2.5ms 

0